In [222]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
import numpy as np
import json

In [223]:
# Define hyperparameters
input_vocab_size = 115 # Vocabulary size for the source language
output_vocab_size = 82 # Vocabulary size for the target language
embedding_dim = 100  # Dimension of the GloVe embeddings
hidden_units = 32 # Number of units in LSTM layers
sequence_length = 37 # Maximum sequence length
batch_size = 32 # Batch size
num_epochs = 200 # Number of training epochs

In [224]:
glove_path = 'GloVe/glove.6B.100d.txt'  # Adjust the path to your downloaded GloVe file
embedding_matrix = {}  # Create an empty dictionary to store the embeddings

In [225]:
with open(glove_path, 'r', encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_matrix[word] = coefs #(coeff == Vectors)

In [226]:
# Create an embedding matrix for the source and target languages
source_embedding_matrix = np.zeros((input_vocab_size, embedding_dim))
target_embedding_matrix = np.zeros((output_vocab_size, embedding_dim))

In [227]:
##load the dataset
with open('card.json', 'r') as json_file:
    dataset = json.load(json_file)

In [228]:
#mapping input and output sequences to integers 
#input sequence = source_tokenizer
#output sequence = target_tokenizer

index_s = 2
index_t = 3
source_tokenizer = {'<PAD>':0,
                   '<start>':1}
target_tokenizer = {'<PAD>':0,
                   '<start>':1,
                   '<end>':2}


for i in dataset:
    question_toks = i["question_toks"]
    # print(query_toks)
    for j in question_toks:
        # print(j)
        
        if j not in source_tokenizer:
            source_tokenizer[j] = index_s
            index_s += 1
# print((frequency))
for i in dataset:
    query_toks = i["query_toks"]
    # print(query_toks)
    for j in query_toks:
        # print(j)
        
        if j not in target_tokenizer:
            target_tokenizer[j] = index_t
            index_t += 1

In [229]:
source_tokenizer

{'<PAD>': 0,
 '<start>': 1,
 'How': 2,
 'many': 3,
 'accounts': 4,
 'do': 5,
 'we': 6,
 'have': 7,
 '?': 8,
 'Count': 9,
 'the': 10,
 'number': 11,
 'of': 12,
 '.': 13,
 'Show': 14,
 'ids': 15,
 ',': 16,
 'customer': 17,
 'names': 18,
 'for': 19,
 'all': 20,
 'What': 21,
 'are': 22,
 'account': 23,
 'and': 24,
 'other': 25,
 'details': 26,
 'with': 27,
 'name': 28,
 '338': 29,
 'is': 30,
 'first': 31,
 'last': 32,
 'phone': 33,
 '162': 34,
 'Give': 35,
 'full': 36,
 'who': 37,
 'has': 38,
 'does': 39,
 'Art': 40,
 'Turcotte': 41,
 'Return': 42,
 'that': 43,
 'each': 44,
 'there': 45,
 'id': 46,
 'most': 47,
 'how': 48,
 'this': 49,
 'person': 50,
 'least': 51,
 'fewest': 52,
 'customers': 53,
 'without': 54,
 'an': 55,
 'not': 56,
 'any': 57,
 'distinct': 58,
 'hold': 59,
 'phones': 60,
 'email': 61,
 'Aniyah': 62,
 'Feest': 63,
 'cards': 64,
 'card': 65,
 'type': 66,
 'codes': 67,
 'numbers': 68,
 'types': 69,
 'date': 70,
 'valid': 71,
 'from': 72,
 'to': 73,
 "'4560596484842": 74,
 

In [230]:
target_tokenizer_rev = {v: k for k, v in target_tokenizer.items()}
target_tokenizer

{'<PAD>': 0,
 '<start>': 1,
 '<end>': 2,
 'SELECT': 3,
 'count': 4,
 '(': 5,
 '*': 6,
 ')': 7,
 'FROM': 8,
 'Accounts': 9,
 'account_id': 10,
 ',': 11,
 'customer_id': 12,
 'account_name': 13,
 'other_account_details': 14,
 'WHERE': 15,
 '=': 16,
 '``': 17,
 '338': 18,
 "''": 19,
 'T2.customer_first_name': 20,
 'T2.customer_last_name': 21,
 'T2.customer_phone': 22,
 'AS': 23,
 'T1': 24,
 'JOIN': 25,
 'Customers': 26,
 'T2': 27,
 'ON': 28,
 'T1.customer_id': 29,
 'T2.customer_id': 30,
 'T1.account_name': 31,
 '162': 32,
 'Art': 33,
 'AND': 34,
 'Turcotte': 35,
 'GROUP': 36,
 'BY': 37,
 'ORDER': 38,
 'DESC': 39,
 'LIMIT': 40,
 '1': 41,
 'ASC': 42,
 'NOT': 43,
 'IN': 44,
 'customer_first_name': 45,
 'customer_last_name': 46,
 'EXCEPT': 47,
 'T1.customer_first_name': 48,
 'T1.customer_last_name': 49,
 'DISTINCT': 50,
 'customer_phone': 51,
 'customer_email': 52,
 'Aniyah': 53,
 'Feest': 54,
 'Customers_cards': 55,
 'card_id': 56,
 'card_type_code': 57,
 'card_number': 58,
 'date_valid_from

In [231]:
for word, i in source_tokenizer.items():
    embedding_vector = embedding_matrix.get(word)
    if embedding_vector is not None:
        source_embedding_matrix[i] = embedding_vector

for word, i in target_tokenizer.items():
    embedding_vector = embedding_matrix.get(word)
    if embedding_vector is not None:
        target_embedding_matrix[i] = embedding_vector

In [232]:
question_tokens = [example["question_toks"] for example in dataset]
query_tokens = [example["query_toks"] for example in dataset]

encoder_input_data_nopad = [[1] + [source_tokenizer[word] for word in sentence] for sentence in question_tokens]


decoder_input_data_nopad = [[1] + [target_tokenizer[word] for word in sentence] + [2] for sentence in query_tokens]


In [233]:
# Pad the sequences to a consistent length
def pad_sequences(sequences, max_length):
    padded_sequences = []
    for sequence in sequences:
        if len(sequence) < max_length:
            padded_sequence = sequence + [0] * (max_length - len(sequence))
        else:
            padded_sequence = sequence[:max_length]
        padded_sequences.append(padded_sequence)
    return padded_sequences

# Pad the encoder and decoder inputs
encoder_input_data = pad_sequences(encoder_input_data_nopad, sequence_length)
decoder_input_data = pad_sequences(decoder_input_data_nopad, sequence_length)
encoder_input_data = np.array(encoder_input_data)
decoder_input_data = np.array(decoder_input_data)

In [234]:
decoder_input_data[1]

array([1, 3, 4, 5, 6, 7, 8, 9, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [235]:
encoder_input_data[1]

array([ 1,  9, 10, 11, 12,  4, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])

In [236]:
#decoder_target_data
target_vocab = {
    0 : 0
}

decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]
decoder_target_data[:, -1] = target_vocab[0]

In [237]:
decoder_target_data[1]

array([3, 4, 5, 6, 7, 8, 9, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [238]:
# Define the encoder
encoder_inputs = tf.keras.layers.Input(shape=(sequence_length,))
encoder_embedding = Embedding(input_vocab_size, embedding_dim, weights=[source_embedding_matrix], trainable=False)(encoder_inputs)
encoder_lstm = LSTM(hidden_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [239]:
# Define the decoder
decoder_inputs = tf.keras.layers.Input(shape=(sequence_length,))
decoder_embedding = Embedding(output_vocab_size, embedding_dim, weights=[target_embedding_matrix], trainable=False)(decoder_inputs)
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation='softmax')
output = decoder_dense(decoder_outputs)

In [240]:
# Build and compile the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [241]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 37)]                 0         []                            
                                                                                                  
 input_12 (InputLayer)       [(None, 37)]                 0         []                            
                                                                                                  
 embedding_10 (Embedding)    (None, 37, 100)              11500     ['input_11[0][0]']            
                                                                                                  
 embedding_11 (Embedding)    (None, 37, 100)              8200      ['input_12[0][0]']            
                                                                                            

In [242]:
print(decoder_target_data.shape)
print(encoder_input_data.shape)
print(decoder_input_data.shape)

(80, 37)
(80, 37)
(80, 37)


In [243]:
# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=num_epochs, validation_split=0.2)

Epoch 1/200
2/2 [==============================] - 7s 1s/step - loss: 4.3928 - val_loss: 4.3808
Epoch 2/200
2/2 [==============================] - 0s 75ms/step - loss: 4.3679 - val_loss: 4.3593
Epoch 3/200
2/2 [==============================] - 0s 72ms/step - loss: 4.3421 - val_loss: 4.3361
Epoch 4/200
2/2 [==============================] - 0s 66ms/step - loss: 4.3146 - val_loss: 4.3095
Epoch 5/200
2/2 [==============================] - 0s 71ms/step - loss: 4.2826 - val_loss: 4.2774
Epoch 6/200
2/2 [==============================] - 0s 71ms/step - loss: 4.2440 - val_loss: 4.2373
Epoch 7/200
2/2 [==============================] - 0s 68ms/step - loss: 4.1982 - val_loss: 4.1858
Epoch 8/200
2/2 [==============================] - 0s 60ms/step - loss: 4.1407 - val_loss: 4.1187
Epoch 9/200
2/2 [==============================] - 0s 63ms/step - loss: 4.0692 - val_loss: 4.0313
Epoch 10/200
2/2 [==============================] - 0s 77ms/step - loss: 3.9828 - val_loss: 3.9186
Epoch 11/200
2/2 [===

Epoch 84/200
2/2 [==============================] - 0s 75ms/step - loss: 1.5592 - val_loss: 1.4126
Epoch 85/200
2/2 [==============================] - 0s 57ms/step - loss: 1.5497 - val_loss: 1.4097
Epoch 86/200
2/2 [==============================] - 0s 60ms/step - loss: 1.5415 - val_loss: 1.4051
Epoch 87/200
2/2 [==============================] - 0s 56ms/step - loss: 1.5346 - val_loss: 1.4025
Epoch 88/200
2/2 [==============================] - 0s 60ms/step - loss: 1.5284 - val_loss: 1.4009
Epoch 89/200
2/2 [==============================] - 0s 57ms/step - loss: 1.5183 - val_loss: 1.3981
Epoch 90/200
2/2 [==============================] - 0s 64ms/step - loss: 1.5118 - val_loss: 1.4008
Epoch 91/200
2/2 [==============================] - 0s 58ms/step - loss: 1.5038 - val_loss: 1.3997
Epoch 92/200
2/2 [==============================] - 0s 57ms/step - loss: 1.4950 - val_loss: 1.3989
Epoch 93/200
2/2 [==============================] - 0s 53ms/step - loss: 1.4892 - val_loss: 1.3934
Epoch 94/2

2/2 [==============================] - 0s 66ms/step - loss: 1.0501 - val_loss: 1.3525
Epoch 167/200
2/2 [==============================] - 0s 58ms/step - loss: 1.0479 - val_loss: 1.3627
Epoch 168/200
2/2 [==============================] - 0s 58ms/step - loss: 1.0434 - val_loss: 1.3560
Epoch 169/200
2/2 [==============================] - 0s 59ms/step - loss: 1.0410 - val_loss: 1.3535
Epoch 170/200
2/2 [==============================] - 0s 60ms/step - loss: 1.0362 - val_loss: 1.3602
Epoch 171/200
2/2 [==============================] - 0s 55ms/step - loss: 1.0290 - val_loss: 1.3598
Epoch 172/200
2/2 [==============================] - 0s 47ms/step - loss: 1.0258 - val_loss: 1.3614
Epoch 173/200
2/2 [==============================] - 0s 58ms/step - loss: 1.0223 - val_loss: 1.3509
Epoch 174/200
2/2 [==============================] - 0s 65ms/step - loss: 1.0181 - val_loss: 1.3532
Epoch 175/200
2/2 [==============================] - 0s 64ms/step - loss: 1.0110 - val_loss: 1.3642
Epoch 176/200


In [244]:
model = tf.keras.Sequential()

In [245]:
dummy_input = np.array([ 8,  9, 10, 11,  3, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0])
_ = model(dummy_input)

In [246]:
##saving model
model.save('seq2seq_model.h5')

In [247]:
from tensorflow.keras.models import load_model
model = load_model('seq2seq_model.h5')


In [259]:
# Load your tokenizers, embeddings, and model as described in the previous response

# Define the start token and initial decoder input
start_token = target_tokenizer['<start>']  # Replace with your actual start token
initial_decoder_input = np.zeros((1, 37))
initial_decoder_input[0, 0] = start_token


In [258]:
# User input
user_question = "Count the total number of accounts"
user_question_tokens = user_question.split()
user_question_sequence = [1] + [source_tokenizer.get(word,0) for word in user_question_tokens]
user_question_sequence = pad_sequences([user_question_sequence], sequence_length)

# Initialize variables for prediction
predicted_sql_tokens = []
current_input = initial_decoder_input

while len(predicted_sql_tokens) < sequence_length:
    predictions = model.predict([user_question_sequence, current_input])
    predicted_token_index = np.argmax(predictions[0, -1, :])
    
    # Convert the token index to the corresponding word
    predicted_word = [key for key, value in target_tokenizer.items() if value == predicted_token_index][0]
    predicted_sql_tokens.append(predicted_word)
    
    # Prepare the input for the next prediction
    current_input = np.array([[predicted_token_index]])
    
    
    # Check for the end token
    if predicted_word == '<end>':  # Replace with your actual end token
        break

# Convert Predicted Tokens to SQL Query
predicted_sql_query = ' '.join(predicted_sql_tokens)

# Execute or Display the SQL Query as needed

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 1) + inhomogeneous part.

In [250]:
user_question_tokens

['Count', 'the', 'total', 'number', 'of', 'accounts']

In [251]:
print(predictions)

[[[  1   9  10 104  11  12   4   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0]]]


In [252]:
print(user_question_sequence)

[[1, 9, 10, 104, 11, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [253]:
print(predicted_word)

SELECT
